In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('D:\\data cleaning\\datasets\\Airbnb\\airbnb.csv')
df

,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,hourse_rules,img_links,price
0,49849504,Perla bungalov,4.71,64,Entire bungalow hosted by Mehmetcan,357334205.0,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,"₹8,078 per night"
1,50891766,Authentic Beach Architect Sheltered Villa with...,New,NaN,Entire villa hosted by Fatih,386223873.0,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,"₹4,665 per night"
2,50699164,cottages sataplia,4.85,68,Entire cabin hosted by Giorgi,409690853.0,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,"₹5,991 per night"
3,49871422,Sapanca Breathable Bungalow,5.0,13,Entire bungalow hosted by Melih,401873242.0,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,"₹11,339 per night"
4,51245886,Bungalov Ev 2,New,NaN,Entire bungalow hosted by Arp Sapanca,414884116.0,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,"₹6,673 per night"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,44697740,"Brown Dome House No. 3, full of healing",4.94,16,Dome hosted by Aidan,161552316.0,"Imhoe-myeon, Jindo, South Jeolla Province, Sou...","4 guests,8 bathrooms","Kitchen,Wifi,Free parking on premises,Private ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠁,C...","Check-in: 3:00 pm - 11:00 pm,Check out: 11:00 ...",https://a0.muscache.com/im/pictures/miso/Hosti...,"₹4,288 per night"
14452,29975283,Bubble Hotel Ubud with Bathtub under the Sky,4.67,6,Tent hosted by Martha,118954227.0,"Gianyar, Indonesia","2 guests,1 bedroom,1 bed,Toilet with sink","Wifi,Free on-street parking,Air conditioning,G...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 3:00 pm - 10:00 pm,Check out: 12:00 ...",https://a0.muscache.com/im/pictures/5ba944b2-b...,"₹15,200 per night"
14453,32977697,Paradise Eco Hub,4.67,3,Private room in dome hosted by Silas,175214568.0,"Kabale, Uganda","2 guests,1 bedroom,2 beds,1 private bathroom","Wifi,Breakfast,Long-term stays allowed,Unavail...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...",Check-in: After 3:00 pm,https://a0.muscache.com/im/pictures/1f442c84-4...,"₹9,103 per night"
14454,8006146,WisDOME - Mentigi Bay Village - Gilis - Lombok,4.86,7,Dome hosted by Cempaka,718423.0,"Gunung Sari, Nusa Tenggara Barat, Indonesia","8 guests,2 bedrooms,4 beds,2.5 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: Flexible,Check out: 12:00 pm,No pets...",https://a0.muscache.com/im/pictures/105116799/...,"₹42,262 per night"


## **Removing columns**

In [ ]:
del df['img_links']
del df['host_id']


## **Checking for Null Values**

In [ ]:
df.isnull().sum()

In [ ]:
# removing the null value in various columns
df.dropna(subset = ['name','safety_rules', 'address']).isnull().sum() # name and host_name was the same so got removed together similiarly safety rules


In [ ]:
# NULL amenities were filled by the 0th index of the mode of all the amenities
df['amenities'].fillna(df['amenities'].mode()[0], inplace = True)
df.isnull().sum()

In [ ]:
# now trying to see for the reviews realised that wherever there is NEW in Rating, possibilty of having NULL in reviews is high but we have
# 3 possible outcomes in reviews ( only where there is NEW in ratings) which are NULL,1,2,3. So 1,2,3 are ints so no need to adjust those but
# replacing Null with 0 seems logical.

df['reviews'] = df['reviews'].fillna('0')
df = df.dropna(subset = ['name','safety_rules','address'])
df.isnull().sum()




## **Searching and removing duplicates**

In [ ]:
# search for duplicates and remove if any
df[df.duplicated()]


df.drop_duplicates(subset = 'id', inplace = True)
len(df)

## **Adding Columns**

#### **Country Column**

In [ ]:
##### Making country column


countries = []

for ad in df['address']:
  if type(ad) == str:
    countries.append(ad.split(',')[-1])
  else:
    countries.append("Unknown")

df['country'] = countries
df

In [ ]:
df[df['country'] == 'Unknown']['country']

### **Extracting Different Features**

In [ ]:
df['features'].head(1000).unique()


**We realised that there is a need to clean this but this is not easy because for ex guest its either guest or guests and we are not even sure that the sequence is same or not or even guest is written or not we have to consider all the problems and then extract it.**

#### **Guest Column**

In [ ]:
#### Adding guests column



guests = []

flag = False

for i in df['features']:
  for j in i.split(','):
    if 'guest' in j:
      guests.append(j.split(' ')[0])
      flag = True

  if flag == False:
   guests.append('1')

df['guests'] = guests
df

#### **Bathrooms Column**

In [ ]:
### Adding bathrooms column



bathrooms = []

for i in df['features']:
  flag  = False
  for j in i.split(','):
    if 'bath' in j:
      bathrooms.append(j.split(' ')[0])
      flag = True
    elif 'toilet' in j:
      bathrooms.append('0')
      flag = True
    elif 'Toilet' in j:
      bathrooms.append('0')
      flag = True


  if flag == False:
   bathrooms.append('1')

df['bathrooms'] = bathrooms
df



#### **Toilets Column**

In [ ]:
### Adding toilets column


toilets = []

for i in df['features']:
  flag  = False
  for j in i.split(','):
    if 'toilet' in j:
      toilets.append('1')
      flag = True
    elif 'Toilet' in j:
      toilets.append('1')
      flag = True


  if flag == False:
   toilets.append('0')

df['toilets'] = toilets
df

#### **Price Column**

In [ ]:
### Changing price column



df['price'] = df['price'].apply(lambda x: x.replace('₹','').replace(',', '').split(' ')[0])
df['price'] = df['price'].astype(int)
df

#### **Studios Column**

In [ ]:
# Adding studios column



studios = []

for i in df['features']:
  if 'Studio' in i.split(','):
    studios.append(1)
  else:
    studios.append(0)

df['studios'] = studios

#### **Bedrooms Column**

In [ ]:
#### Making bedrooms column


bedrooms = []


for i in df['features']:
  flag = False
  for j in i.split(','):
    if 'bedroom' in j:
      bedrooms.append(j.split(' ')[0])
      flag = True
  if flag == False:
    bedrooms.append('0')
df['bedrooms'] = bedrooms
df

#### Copying DF

In [ ]:
# Making a copy of the df so that after making the bedrooms coulumn we can delete the bedroom word from the features series to find the beds column
df_1 = df.copy()


#### **Beds Column**

In [ ]:
# Deleting the word bedrroms from the features series and then making the beds column

df_1['features'] = df_1['features'].apply(lambda x: x.replace('bedroom','').replace('bedrooms', '').split(','))

beds = []

for i in df_1['features']:
  flag = False
  for j in i:
    if 'bed' in j:
      beds.append(j.split(' ')[0])
      flag = True
  if flag == False:
    beds.append('0')
df_1['beds'] = beds
df_1





In [ ]:
# Loading the beds column in the original df

df['beds'] = df_1['beds']
df

#### **Checkin Column**

In [ ]:
# Removing the word Check-in from the house_rules column
df_1['hourse_rules'] = df_1['hourse_rules'].apply(lambda x: x.replace('Check-in:','').split(','))



In [ ]:
# Making Checkin column in the main df
checkin = []

for i in df_1['hourse_rules']:
  checkin.append(i[0])

df['checkin'] = checkin
df

#### **Checkout Column**

In [ ]:
# Removing the word Check out from the house_rules column
df_1 = df.copy()
df_1['hourse_rules'] = df_1['hourse_rules'].apply(lambda x: x.replace('Check out:','').split(','))




In [ ]:
#### Adding checkout column

checkout = []

for i in df_1['hourse_rules']:
  if len(i) < 2:
    checkout.append(None)
  else:
    checkout.append(i[1])

df['checkout'] = checkout
df

## **Final Airbnb_v3.csv**

In [1]:
s = 'hi'
s.

HELlo
